In [13]:
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

In [14]:
# 디렉토리 재설정
try:
    os.chdir("C:\\Users\\SAMSUNG\\Desktop\\나\\블로그\\웹스크래핑\\") 
    print("Directory changed")
except OSError: # 예외처리
    print("Can't change the Current Working Directory")

Directory changed


In [15]:
from selenium import webdriver
options=webdriver.ChromeOptions()
options.add_argument('--privileged')
options.add_argument('headless')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--start-maximized')
options.add_argument('disable-gpu')  # GPU 사용 안함
options.add_argument('lang=ko_KR')  # 언어 설정

In [21]:
def web_scraping(code, start_page,end_page, count):
    title_result=[]
    date_result=[]
    like_result=[]
    dislike_result=[]
    views_result=[]
    page=2

    driver=webdriver.Chrome("chromedriver.exe",chrome_options=options)
    url="https://finance.naver.com/item/board.nhn?code="+code+"&page="+str(start_page)
    driver.get(url)
    while (start_page<=end_page):
        start_page+=1
        html = driver.page_source
        soup=bs(html,'lxml')

        title_css_selector="#content > div.section.inner_sub > table.type2 > tbody > tr > td.title > a"
        title=soup.select(title_css_selector)
        for i in title:
            title_result.append(i.text.replace('\t','').replace('\n',''))
    
        date_css_selector="#content > div.section.inner_sub > table.type2 > tbody > tr > td:nth-of-type(1) > span"
        date=soup.select(date_css_selector)
        for j in date:
            date_result.append(j.text)
    
        views_css_selector="#content > div.section.inner_sub > table.type2 > tbody > tr > td:nth-of-type(4) > span"
        views=soup.select(views_css_selector)
        for k in views:
            views_result.append(k.text)
    

        like_css_selector="#content > div.section.inner_sub > table.type2 > tbody > tr > td:nth-of-type(5) > strong"
        like=soup.select(like_css_selector)
        for l in like:
            like_result.append(l.text)
    
        dislike_css_selector="#content > div.section.inner_sub > table.type2 > tbody > tr > td:nth-of-type(6) > strong"
        dislike=soup.select(dislike_css_selector)
        for m in dislike:
            dislike_result.append(m.text)
        
        if (page>10):
            nexttb_css_selector=f"#content > div.section.inner_sub > table:nth-child(3) > tbody > tr > td:nth-child(2) > table > tbody > tr > td.pgR > a"
            nexttb_element=driver.find_element_by_css_selector(nexttb_css_selector)
            nexttb_element.click()
            page=2
            continue
    
        nextpg_css_selector=f"#content > div.section.inner_sub > table:nth-child(3) > tbody > tr > td:nth-child(2) > table > tbody > tr > td:nth-child({page}) > a"
        nextpg_element=driver.find_element_by_css_selector(nextpg_css_selector)
        nextpg_element.click()
        page+=1

    concating(code, title_result,date_result,views_result,like_result,dislike_result,count)
    driver.quit()

In [19]:
def concating(code, title_result,date_result,views_result,like_result,dislike_result,count):
    concat_data={'title':title_result,
                'date':date_result,
                'views':views_result,
                'like':like_result,
                'dislike':dislike_result}
    scraping_df=pd.DataFrame(concat_data)
    scraping_df.to_csv('./title_crwaling_'+code+'_'+str(count)+'.csv',mode='w',encoding='utf-8-sig',header=True,index=True)

In [22]:
if __name__ == '__main__':
    code=input("기업 코드를 입력해주세요.: ")
    start_page=int(input("첫 번째 페이지를 입력해주세요.: "))
    end_page=int(input("마지막 페이지를 입력해주세요.: "))
    count=1
    while (start_page<=end_page):
        if (end_page-start_page<100):
            web_scraping(code,start_page,end_page,count)
        else:
            web_scraping(code,start_page,start_page+100,count)
        print("%d번째 스크래핑이 끝났습니다."%count)
        start_page+=101
        count+=1

기업 코드를 입력해주세요.: 005930
첫 번째 페이지를 입력해주세요.: 1
마지막 페이지를 입력해주세요.: 200


C:\Users\SAMSUNG\anaconda3\envs\project\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


1번째 스크래핑이 끝났습니다.
2번째 스크래핑이 끝났습니다.
